In [1]:
import json
import os
import sys
import warnings

import numpy as np
import pandas as pd
from datetime import datetime
from pprint import pprint
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

In [2]:
sys.path.append(os.path.join('..', 'src'))

In [3]:
import importlib
import model
importlib.reload(model)

from model import get_model_params, timer, measure_prediction_time, apply_ml_model, save_model_parameters, save_model_metrics

# set model parameters and capture data

In [4]:
# scoring = 'neg_mean_squared_error'
scoring = 'f1'

inputs = os.path.join('..', 'data', '03_processed')
models_reports = os.path.join('..', 'data', '04_models')
model_outputs = os.path.join('..', 'data', '05_model_output')
reports = os.path.join('..', 'data', '06_reporting')

X_train            = pd.read_csv(os.path.join(inputs, 'X_train.csv'), index_col='id')
X_train_onehot         = pd.read_csv(os.path.join(inputs, 'X_train_onehot.csv'), index_col='id')
y_train            = pd.read_csv(os.path.join(inputs, 'y_train.csv'), index_col='id')

data_list = [X_train, X_train_onehot, y_train]

for df in data_list:
    print(df.shape)

(559, 13)
(559, 20)
(559, 1)


In [5]:
X_train_onehot.head()

,anomaly_count_lag10,cycle,dummy_preset_1_2,dummy_preset_1_3,dummy_preset_2_2,dummy_preset_2_3,dummy_preset_2_4,dummy_preset_2_5,dummy_preset_2_6,dummy_preset_2_7,dummy_preset_2_8,frequency,lag_1,lag_2,lag_3,pressure,temperature,vibrationx,vibrationy,vibrationz
id,,,,,,,,,,,,,,,,,,,,
0,0.0,1.0,0,1,0,0,0,0,1,0,0,44.483250,0,0,0,47.657254,44.235186,46.441769,64.820327,66.454520
1,0.0,2.0,1,0,0,0,1,0,0,0,0,60.228715,0,0,0,63.172076,60.807234,62.005951,80.714431,81.246405
2,0.0,3.0,1,0,0,0,0,0,0,0,0,80.993479,0,0,0,83.032190,79.027536,82.642110,98.254386,98.785196
3,0.0,4.0,1,0,0,1,0,0,0,0,0,80.315567,0,0,0,100.508634,79.716242,122.362321,121.363429,118.652538
4,0.0,5.0,1,0,0,0,0,1,0,0,0,64.245166,0,0,0,51.764833,39.989054,42.514302,61.037910,50.716469


# Machine Learning

convergence warning: https://stackoverflow.com/questions/20681864/lasso-on-sklearn-does-not-converge

In [6]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

In [7]:
ml_dict = {}

# Specify the hyperparameter space
# if target_type == 'regression':
#     parameters = {
#     'model__alpha': np.linspace(0.2, 1, 5), 
#     'model__l1_ratio': np.linspace(0, 1, 5),
#     'model__random_state':[42]
#     }
#     ml_model = ElasticNet()
#     # set tol, default is 1e-4
#     do_transform_label = 'log'
# elif target_type == 'binary':
c_space = np.logspace(-5, 1, 5)
parameters = {
'model__C': c_space, 
'model__penalty': ['l2'],
'model__random_state':[42]
}
ml_model = LogisticRegression()
do_transform_label = None

# key = 'standard'

## test with different preprocessing steps
There are 2 different X_sets: On X_train_onehot, I applied one-hot encoding, while on X_train I applied Ordinal Encoding. The former is aimed at linear regression models, and the later is generally used for tree models.

On 'column' parameter, I am able to choose column groups. For instance, I might exclude collinear variables obtained from the VIF function applied on notebook 5. That is useful for linear regression models.

```python
treat_collinearity = False, do_build_polynomals=False, do_treat_skewness=False
```

In [8]:
model_type = 'reg'
ml_dict[model_type] = {}
columns = X_train_onehot.columns

clf, ml_dict[model_type]['train_time'], ml_dict[model_type]['prediction_time'] = apply_ml_model(
    X_train_onehot, y_train, columns, ml_model, parameters, scoring,
    do_build_polynomals=False, 
    do_treat_skewness=False,
    imputation=Imputer(strategy='median'), scaler=StandardScaler(),
    )
ml_dict[model_type]['best_params'], ml_dict[model_type]['best_score']  = get_model_params(clf, scoring)
pprint(ml_dict)

save_model_parameters(models_reports, model_type, clf)
save_model_metrics(model_outputs, model_type, ml_dict)

test type: False
{'reg': {'best_params': {'model__C': 0.31622776601683794,
                         'model__penalty': 'l2',
                         'model__random_state': 42},
         'best_score': 0.5520445344129555,
         'prediction_time': 0.0003005,
         'train_time': 0.548985}}


```python
treat_collinearity = True, do_build_polynomals=False, do_treat_skewness=False,
```

In [9]:
model_type = 'reg_nocol'
ml_dict[model_type] = {}

# columns_nocol = dfs_dict['X_train_oh_nocol'].columns.to_list()

clf, ml_dict[model_type]['train_time'], ml_dict[model_type]['prediction_time'] = apply_ml_model(
    X_train_onehot, y_train, columns, ml_model, parameters, scoring,
    do_build_polynomals=False, 
    do_treat_skewness=False,
    imputation=Imputer(strategy='median'), scaler=StandardScaler(),
    )
ml_dict[model_type]['best_params'], ml_dict[model_type]['best_score']  = get_model_params(clf, scoring)
pprint(ml_dict)

save_model_parameters(models_reports, model_type, clf)
save_model_metrics(model_outputs, model_type, ml_dict)

test type: False
{'reg': {'best_params': {'model__C': 0.31622776601683794,
                         'model__penalty': 'l2',
                         'model__random_state': 42},
         'best_score': 0.5520445344129555,
         'prediction_time': 0.0003005,
         'train_time': 0.548985},
 'reg_nocol': {'best_params': {'model__C': 0.31622776601683794,
                               'model__penalty': 'l2',
                               'model__random_state': 42},
               'best_score': 0.5520445344129555,
               'prediction_time': 0.0002999,
               'train_time': 0.507}}


I might use the alternative encoding just to demonstrate the impact on the score.